# Depletion
This session is intended to introduce you to the depletion interface/solver contained in OpenMC. In this session, we will model a simple fuel pin in an infinite lattice using the Python API and then build and examine some of the necessary components for performing depletion analysis.

In [ ]:
import math
import matplotlib.pyplot as plt
import openmc.deplete

## Build the Geometry

For this demonstration, we will use a simple LWR fuel pin. Note that when running depletion simulations, we need to use the `Model` class:

To ensure our geometry looks right, let's plot it.

For any materials that are depleted, we have to assign the volume explicitly:

## The `Chain` class

The OpenMC depletion interface can be accessed from the `openmc.deplete` module, and has a variety of classes that will help us. In order to run the depletion calculation we need the following information:

1. Decay modes and half-lives, fission product yields (FPYs), and reaction data
2. Operational power, power density, or source rate (for fixed source calculations)
3. Desired depletion schedule
4. Desired time integration scheme

The additional nuclear data (decay and FPY) are stored in a "depletion chain" XML file that is generated from a combination of the `IncidentNeutron`, `Decay`, and `FissionProductYields` classes we saw yesterday. The `openmc.deplete.Chain` class to aggregate the necessary data. While the `Chain` class has a `from_endf` method that can be used, you may prefer to download pre-generated XML-representations instead at https://openmc.org/depletion-chains/.

For this session, we will be using a much smaller depletion chain that contains very few nuclides that results in shorter simulations.

## The Depletion `CoupledOperator`

The primary entry point for depletion is the `openmc.deplete.CoupledOperator` class. It relies on the `openmc.deplete.Chain` and helper classes to run `openmc`, retrieve and normalize reaction rates, and other perform other tasks. For a thorough description, please see the full API documentation.

We will create our CoupledOperator using the geometry and settings from above, and our simple chain file. The materials are read in automatically using the `materials.xml` file.

## Analyzing Results

The depletion simulation produces a few output files. First, the statepoint files from each individual transport simulation are written to `openmc_simulation_n<N>.h5`, where `<N>` indicates the current depletion step. Any tallies that we defined in `tallies.xml` will be included in these files across our simulations. We have 7 such files, one for each our of 6 depletion steps and the initial state.

## Depletion Integrators

Using higher-order integrators, like the `CECMIntegrator`, `EPCRK4Integrator` with a fourth order Runge-Kutta, or the `LEQIIntegrator`, can improve the accuracy of a simulation, or at least allow you to take longer depletion steps between transport simulations with similar accuracy.

## Integration timestep/power options

## Reaction rate / fission product yield modes

By default, OpenMC will determine reaction rates for all transmutation reactions by setting up an tallies for each nuclide/reaction, which can be quite expensive if there are many materials, each with hundreds of nuclides. One alternative is to do a "flux collapse" method, whereby a multigroup flux spectrum is tallied and then *a posteriori* collapsed with the cross sections to determine reaction rates. This is enabled with the `reaction_rate_mode` and `reaction_rate_opts` arguments to `CoupledOperator`.

OpenMC also gives a few different ways of specifying what fission product yields to use. By default, it will use thermal FPYs, but this can be changed with the `fission_yield_mode` and `fission_yield_opts` arguments to `CoupledOperator`.

## Normalization (energy) modes

By default, the depletion chain has a set of fixed Q values that are used to estimate energy deposited from fission. However, this doesn't account for energy redistribution/loss from coupled neutron-photon transport or any incident neutron energy-dependent effects. Thus, you should also be aware that OpenMC gives you knobs to turn for how reaction rates are normalized based on observed energy deposition:

```Python
openmc.deplete.CoupledOperator(..., normalization_mode="fission-q")  # fixed Q values for fission
openmc.deplete.CoupledOperator(..., normalization_mode="energy-deposition")  # explicit heating tallies
openmc.deplete.CoupledOperator(..., normalization_mode="source-rate")  # for fixed source (activation) calculations
```

## Reducing a depletion chain

A lot of the computational cost of running depletion has to do with the size and complexity of the depletion chain. OpenMC allows you to "reduce" a depletion chain to a select set of nuclides/pathways which can be very useful if you are only interested in selected nuclides (e.g., activation in a specific material). This is enabled by the `reduce()` method of the `Chain` class. Let's start with a full-blown depletion chain based on ENDF/B-VII.1 and then reduce it to a select set of nuclides for the sake of analyzing activation in nickel.

## Matching results with other codes

Getting OpenMC results to match with other codes (e.g., MCNP or Serpent) can be very challenging because of all the inputs that go in to the code. This is especially true for depletion, where we are now concerned not only with getting the transport solution to match but also the depletion integration methods, decay/FPY data, approximations, etc. In order to get a perfect match, you'll need to ensure:

- The model geometry/material definitions are equivalent
- The same neutron cross sections are being used (e.g., ENDF/B-VII.1)
- Same treatment for thermal scattering / probability tables
- Same decay data (depletion chain)
- Same fission product yields (depletion chain)
- Same fission product yield interpolation method (`CoupledOperator(fission_yield_mode=...)`)
- Same fission Q values (`CoupledOperator(fission_q=...)`)
- Same depletion integration method (`PredictorIntegrator`, `CECMIntegrator`, etc.)

Even when all of these are the same, it's still entirely possible to see differences because of other appoximations in the codes that cannot be changed.

## Choice of depletion step size

A general rule of thumb is to use depletion step sizes around 2 MWd/kgHM, where kgHM is really the initial heavy metal mass in kg. If your problem includes integral burnable absorbers, these typically require shorter time steps at or below 1 MWd/kgHM. These are typically valid for the predictor scheme, as the point of recent schemes is to extend this step size. A good convergence study, where the step size is decreased until some convergence metric is satisfied, is a beneficial exercise.

We can use the `CoupledOperator` to determine our maximum step size using this recommendation. The `heavy_metal` attribute returns the mass of initial heavy metal in g, which, using our power, can be used to compute this step size. $$\frac{2\,MWd}{kgHM} = \frac{P\times\Delta}{hm_{op}}$$

## Differentiating burnable materials

OpenMC allows you to differentiate materials that reappear in multiple places. If we had built an entire core with the single `fuel` material, every pin would be depleted using the same averaged spectrum and reaction rates, which is physically incorrect. The `CoupledOperator` can differentiate these materials using the `diff_burnable_mats` argument. In the case, the volume assigned to the original material will be divided over each new instance of the material.

In [ ]:
new_model = openmc.Model()
new_model.settings = model.settings

# Create materials for fuel, clad, and water
fuel = openmc.Material(name="uo2")
fuel.add_element("U", 1, percent_type="ao", enrichment=4.25)
fuel.add_element("O", 2)
fuel.set_density("g/cc", 10.4)
fuel.volume = 4 * math.pi * radii[0] ** 2

clad = openmc.Material(name="clad")
clad.add_element("Zr", 1)
clad.set_density("g/cc", 6)

water = openmc.Material(name="water")
water.add_element("O", 1)
water.add_element("H", 2)
water.set_density("g/cc", 1.0)
water.add_s_alpha_beta("c_H_in_H2O")
model.materials = openmc.Materials([fuel, clad, water])

# Create a fuel pin universe
radii = [0.42, 0.45]
pin_surfaces = [openmc.ZCylinder(r=r) for r in radii]
materials = [fuel, clad, water]
pin_univ = openmc.model.pin(pin_surfaces, model.materials)

lattice = openmc.RectLattice()
lattice.lower_left = (-pitch, -pitch)
lattice.pitch = (pitch, pitch)
lattice.universes = [
    [pin_univ, pin_univ],
    [pin_univ, pin_univ]
]
lattice.outer = pin_univ

# Put it into a single cell defined as a box with reflective boundary conditions
bound_box = openmc.rectangular_prism(2*pitch, 2*pitch, boundary_type="reflective")
root_cell = openmc.Cell(fill=lattice, region=bound_box)
new_model.geometry = openmc.Geometry([root_cell])

new_model.geometry.root_universe.plot(width=(2*pitch, 2*pitch))